In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
test_data = {}
# blocks 1 and 3 are gain and have 4 for win and 3 for loss
# blocks 2 and 4 are loss and have 2 for win and 1 for loss
for i in range(1,101):
    testData = spio.loadmat('genData_smG_rlG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    block = [
                {
                    'action': np.array([[resp-1 for ([id],[resp]) in zip(struct[0,0]['blockID'],struct[0,0]['selectStimIndex']) if id==blockID]]),
                    'state': np.array([[stimOffers for ([id],stimOffers) in zip(struct[0,0]['blockID'],struct[0,0]['stimOffers']) if id==blockID]]),
                    'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew']) if id==blockID]]),
                    'id': 'S'+str(i),

                    'block': blockID
                } for blockID in np.unique([struct[0,0]['blockID']])

            ]
#     block = [
#                 {
#                     'action': np.array([struct[0,0]['selectStimIndex'].flatten()])-1,
#                     'state': np.array([struct[0,0]['stimOffers']]),
#                     'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew'])]]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
    if i<91:
        data['S'+str(i)] = block
    else:
        test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[2, 2, 0, 1, 2, 1, 0, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
           2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
           1, 1, 2, 2]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],
           [1, 2],
           [2, 1],
           [3, 2],
           [3, 2],
           [2, 3],
           [3, 1],
           [2, 3],
           [1, 3],
           [3, 2],
           [2, 1],
           [2, 1],
           [3, 2],
           [1, 3],
           [2, 1],
           [2, 1],
           [2, 3],
           [1, 2],
           [2, 3],
           [3, 2],
           [1, 3],
           [3, 1],
           [3, 2],
           [3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [2, 3

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=3, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/simpleModel_selectedStimulus_rewardOutcome/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-02 14:25:52,840 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:25:52,842 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:25:52,843 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:25:52,844 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:25:52,846 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-02 14:26:06,423 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/simpleModel_selectedStimulus_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:26:31,499 - DeepRL - DEBUG - global iter =    0 total obj: 16472.7361
2021-07-02 14:26:47,324 - DeepRL - DEBUG - global iter =    1 total obj: 11300.8036
2021-07-02 14:27:03,650 - DeepRL - DEBUG - global iter =    2 total obj: 7790.8886
2021-07-02 14:27:19,368 - DeepRL - DEBUG - global iter =    3 total obj: 6135.2244
2021-07-02 14:27:35,061 - DeepRL - DEBUG - global iter =    4 total obj: 5090.0870
2021-07-02 14:27:49,857 - DeepRL - DEBUG - global iter =    5 total obj: 4500.4528
2021-07-02 14:28:06,391 - DeepRL - DEBUG - global iter =    6 total obj: 4066.8503
2021-07-02 14:28:22,629 - DeepRL - DEBUG - global iter =    7 total obj: 3677.5682
2021-07-02 14:28:41,304 - DeepRL - DEBUG - global iter =    8 total obj: 3364.9152
2021-07-02 14:29:00,049 - DeepRL - DEBUG - global iter =    9 total obj: 3168.7296
2021-07-02 14:29:19,680 - DeepRL - DEBUG - global iter =   10 total obj: 3015.8171
2021-07-02 14:29:40,140 - DeepRL - DEBUG - global iter =   11 total obj: 2813.1976
20

INFO:tensorflow:../results/simpleModel_selectedStimulus_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[4, 3, 3, ..., 4, 4, 3],
          [2, 1, 1, ..., 2, 2, 1],
          [4, 3, 4, ..., 3, 4, 4],
          ...,
          [2, 1, 1, ..., 2, 2, 2],
          [4, 3, 4, ..., 3, 4, 4],
          [2, 1, 2, ..., 1, 2, 2]], dtype=int32),
   'action': array([[2, 2, 0, ..., 0, 0, 0],
          [2, 2, 0, ..., 0, 0, 0],
          [0, 0, 2, ..., 1, 2, 2],
          ...,
          [2, 2, 0, ..., 1, 2, 2],
          [0, 0, 2, ..., 1, 2, 2],
          [0, 0, 2, ..., 1, 2, 2]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[1, 2],
           [3, 1],
           [1, 3],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          ...,
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ..

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-02 14:46:09,440 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:46:09,442 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:46:09,443 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:46:09,444 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:46:09,446 - DeepRL - DEBUG - test data-points: 10
2021-07-02 14:46:28,989 - DeepRL - DEBUG - opt started...
2021-07-02 14:46:28,991 - DeepRL - DEBUG - started testing...
2021-07-02 14:48:28,162 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_selectedStimulus_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:49:15,529 - DeepRL - DEBUG - global iter =    0 total obj: 17406.0417
2021-07-02 14:49:33,324 - DeepRL - DEBUG - global iter =    1 total obj: 13298.7534
2021-07-02 14:49:51,015 - DeepRL - DEBUG - global iter =    2 total obj: 10127.4152
2021-07-02 14:50:08,657 - DeepRL - DEBUG - global iter =    3 total obj: 8149.4973
2021-07-02 14:50:25,643 - DeepRL - DEBUG - global iter =    4 total obj: 6976.1712
2021-07-02 14:50:40,185 - DeepRL - DEBUG - global iter =    5 total obj: 6227.2318
2021-07-02 14:50:56,671 - DeepRL - DEBUG - global iter =    6 total obj: 5638.0275
2021-07-02 14:51:10,245 - DeepRL - DEBUG - global iter =    7 total obj: 5194.8610
2021-07-02 14:51:21,039 - DeepRL - DEBUG - global iter =    8 total obj: 4824.5613
2021-07-02 14:51:33,124 - DeepRL - DEBUG - global iter =    9 total obj: 4440.3373
2021-07-02 14:51:33,126 - DeepRL - DEBUG - started testing...
2021-07-02 14:53:35,254 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_selectedStimulus_rewardOutcome/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:54:13,137 - DeepRL - DEBUG - global iter =   10 total obj: 4112.6070
2021-07-02 14:54:30,538 - DeepRL - DEBUG - global iter =   11 total obj: 3851.2693
2021-07-02 14:54:47,933 - DeepRL - DEBUG - global iter =   12 total obj: 3674.5303
2021-07-02 14:55:04,900 - DeepRL - DEBUG - global iter =   13 total obj: 3536.7979
2021-07-02 14:55:21,180 - DeepRL - DEBUG - global iter =   14 total obj: 3412.3306
2021-07-02 14:55:34,415 - DeepRL - DEBUG - global iter =   15 total obj: 3259.0789
2021-07-02 14:55:45,460 - DeepRL - DEBUG - global iter =   16 total obj: 3090.2449
2021-07-02 14:55:57,122 - DeepRL - DEBUG - global iter =   17 total obj: 3024.4219
2021-07-02 14:56:09,396 - DeepRL - DEBUG - global iter =   18 total obj: 2948.7045
2021-07-02 14:56:21,541 - DeepRL - DEBUG - global iter =   19 total obj: 2850.1886
2021-07-02 14:56:21,543 - DeepRL - DEBUG - started testing...
2021-07-02 14:58:48,425 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/simpleModel_selectedStimulus_rewardOutcome/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:04:21,969 - DeepRL - DEBUG - global iter =   30 total obj: 2241.5046
2021-07-02 15:04:33,610 - DeepRL - DEBUG - global iter =   31 total obj: 2481.9163
2021-07-02 15:04:45,200 - DeepRL - DEBUG - global iter =   32 total obj: 2269.2557
2021-07-02 15:04:55,986 - DeepRL - DEBUG - global iter =   33 total obj: 2488.6173
2021-07-02 15:05:05,964 - DeepRL - DEBUG - global iter =   34 total obj: 2216.4348
2021-07-02 15:05:16,442 - DeepRL - DEBUG - global iter =   35 total obj: 2305.5153
2021-07-02 15:05:27,942 - DeepRL - DEBUG - global iter =   36 total obj: 2024.4235
2021-07-02 15:05:42,322 - DeepRL - DEBUG - global iter =   37 total obj: 2141.9752
2021-07-02 15:05:56,632 - DeepRL - DEBUG - global iter =   38 total obj: 2115.0003
2021-07-02 15:06:11,000 - DeepRL - DEBUG - global iter =   39 total obj: 2148.0633
2021-07-02 15:06:11,002 - DeepRL - DEBUG - started testing...
2021-07-02 15:08:26,433 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_selectedStimulus_rewardOutcome/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:08:43,205 - DeepRL - DEBUG - global iter =   40 total obj: 2287.5385
2021-07-02 15:08:54,292 - DeepRL - DEBUG - global iter =   41 total obj: 2039.4642
2021-07-02 15:09:05,416 - DeepRL - DEBUG - global iter =   42 total obj: 2158.3725
2021-07-02 15:09:16,660 - DeepRL - DEBUG - global iter =   43 total obj: 1989.6463
2021-07-02 15:09:27,869 - DeepRL - DEBUG - global iter =   44 total obj: 2077.5372
2021-07-02 15:09:40,567 - DeepRL - DEBUG - global iter =   45 total obj: 2034.9647
2021-07-02 15:09:52,651 - DeepRL - DEBUG - global iter =   46 total obj: 2052.3237
2021-07-02 15:10:05,325 - DeepRL - DEBUG - global iter =   47 total obj: 2082.5277
2021-07-02 15:10:19,258 - DeepRL - DEBUG - global iter =   48 total obj: 1980.6645
2021-07-02 15:10:33,553 - DeepRL - DEBUG - global iter =   49 total obj: 1954.4590
2021-07-02 15:10:33,555 - DeepRL - DEBUG - opt finished.
2021-07-02 15:10:33,557 - DeepRL - DEBUG - started testing...
2021-07-02 15:12:04,823 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/simpleModel_selectedStimulus_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[2, 2, 2, 0, 2, 2, 0, 2, 1, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2,
          0, 2, 2, 1, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
          1, 1, 0, 0]]),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [2, 1],
          [1, 3],
          [2, 1],
          [2, 1],
          [1, 3],
          [3, 2],
          [3, 1],
          [1, 3],
          [2, 3],
          [1, 2],
          [1, 3],
          [2, 1],
          [1, 3],
          [3, 1],
          [1, 2],
          [2, 3],
          [2, 1],
          [3, 1],
          [2, 1],
          [3, 2],
          [1, 2],
          [1, 2],
          [2, 3],
          [1, 3],
          [3, 2],
          [2, 3],
          [3, 1],
          [3, 1],
          [2